# My Assignment - segmenting and clustering neighborhoods in Toronto

#### Install BeautifulSoup4 python module

In [ ]:
!pip install beautifulsoup4

#### Import required modules

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import folium

import requests
from bs4 import BeautifulSoup

#### Retrieve the wikipedia page and make parser using BeautifulSoup

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
raw_page = requests.get(url)
page = raw_page.text
#print(page)
soup = BeautifulSoup(page, 'html.parser')

#### Define get_text() function to collect the text data in the specific html tag

In [ ]:
def get_text(tlist):
    _temp = list()
    for t in tlist:
        _temp.append(t.text.strip())
    return _temp

#### Make the DataFrame
- Find table header and gather names of the columns

In [ ]:
toronto_fsa = soup.find("table",{"class": "wikitable sortable"})
col_headers = toronto_fsa.find_all("th")
columns = get_text(col_headers)
columns

- Find all table cells and gather the values of them
- Then make a DataFrame, 'df' using the column names and values

In [ ]:
val_list = toronto_fsa.find_all("td")
vlist = np.asarray(get_text(val_list))
data_dict = {}
for i, c in enumerate(columns):
    data_dict[c] = vlist[i::3]

df = pd.DataFrame(data_dict)
df

#### Refine the dataframe
- Remove the rows with the value, "Not assigned", in the "Borough" columns
- Merge rows with the same "Postcode"

In [ ]:
df = df[df.Borough != "Not assigned"]
df = df.groupby("Postcode").aggregate({"Borough":"first", "Neighbourhood":lambda x: ",".join(x)}).reset_index()
df

- Fill the "Not assigned" values in "Neighbourhood" column with the values in the "Borough" field

In [ ]:
_tmpdf = df[df.Neighbourhood == "Not assigned"]
for i, v in zip(_tmpdf.index, _tmpdf.values):
    df.iloc[i].Neighbourhood = df.iloc[i].Borough
df